## 20200903T12:00 Jupyter

前面介绍了ssh登录jetson nano的方法。除此以外，还需要配置通过jupyter远程登录jetson nano，控制jetbot小车。

以下几个基本概念，建议提前掌握清楚
- `remote`:远程小车jetson nano，在我这个例子里对应ip是192.168.1.15
- `PC`: 我用于控制jetbot的本地PC，对应192.168.1.4

分成以下几个步骤


注意：本着能用即可的原则，我没有设置ssl安全连接。要提高安全度，可以考虑加入SSL协议，有这方面需求的小伙伴可以自行google

### 1.1 设置密码

登录remote
```shell
$ ssh --port 22 yanyuan@192.168.1.15
```

- `--port 22` 是remote上的ssh端口号，默认值22，如果没有做额外改动的话，这个参数可以省掉
- `yanyuan@192.168.1.15`，`@`后面的ip地址是remote的ip，前面是remote上的用户名，记得改成相应的

在remote上运行2个命令，第1个是生成jupyter的配置文件

```shell
$ jupyter notebook --generate-config
```
生成的配置文件是`~/.jupyter/jupyter_notebook_config.py`，打开它

```shell
$ nano ~/.jupyter/jupyter_notebook_config.py
```

本着最小化改动的原则，修改其中的1行代码，让remote上jupyter默认以9996端口启动(当然你也可以改成自己喜欢的其他端口）

```shell
c.NotebookApp.port = 9996 
```

第2个是设置remote上的jupyter密码，将来我们从`PC`上访问`remote`的jupyter时，是需要的

```shell
$ jupyter notebook password
Enter password:  ****
Verify password: ****
[NotebookPasswordApp] Wrote hashed password to /home/yanyuan/.jupyter/jupyter_notebook_config.json
```
留心最后1行，哈希密码保存的位置，感兴趣的可以打开这个文件看一下（非必需）


### 1.2 remote上运行jupyter

设置好了之后，在`remote`上运行jupyter。可选jupyter notebook或者jupyter lab，二者区别不大（可以google看一下有什么不同）。这里以我常用的jupyter lab为例

```shell
$ jupyter lab --no-browser
[I 10:29:37.632 LabApp] JupyterLab extension loaded from /home/yanyuan/archiconda3/lib/python3.7/site-packages/jupyterlab
[I 10:29:37.633 LabApp] JupyterLab application directory is /home/yanyuan/archiconda3/share/jupyter/lab
[I 10:29:37.645 LabApp] Serving notebooks from local directory: /home/yanyuan
[I 10:29:37.645 LabApp] The Jupyter Notebook is running at:
[I 10:29:37.645 LabApp] http://localhost:9996/
[I 10:29:37.646 LabApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
```

### 1.3 PC上远程访问jupyter

现在回到`PC`，在term里输入
```shell
$ ssh -N -f -L localhost:9906:localhost:9996 yanyuan@192.168.1.15
yanyuan@192.168.1.15's password: 
```
- `ssh`: 远程登录的命令，自己google吧
- `-N`: port forwarding时用的参数
- `-f`: 让ssh在后台运行
- `-L localhost:9906:localhost:9996`: 以`local_socket:remote_socket`的方式设置端口映射。后半段localhost:9996是`remote`上的端口号（如果你设置成了别的，需要相应做改动）。前半段localhost:9906是在`PC`上建立的端口号（也可以改为别的），用于在`PC`远程访问`remote`上的`Jupyter`环境
- `password`:这里是要你输入remote上，这个id的登录密码，不是前面jupyter notework password设置的密码

搞定之后，在`PC`上打开浏览器（chrome或者firefox，随你心意），地址栏敲入网址`http://localhost:9906/`，回车。会要求你输入密码，这就是前面jupyter notework password设置的密码了。

搞定，现在我们已经实现了在`PC`上通过jupyter远程控制`remote`。

### 1.4 懒人攻略
每次登录都要搞这么大一圈，头皮都发麻。为了科学有效合理且理直气壮地偷懒，我们来搞2组快捷键

第1组是在`remote`上的，编辑`.zshrc`或者`.bashrc`文件（取决于你现在用的是bash还是zsh），写入一个程序
```shell
function jpt(){
    # Fires-up a Jupyter notebook by supplying a specific port
    jupyter lab --no-browser --port=$1
}
```

这样每次只需要在`remote`的terminal中输入如下命令，就可以打开Jupyter Lab的server端了
```shell
$ jpt 9996
```

第2组是在`PC`上的，同样编辑`.zshrc`或者`.bashrc`文件（取决于你现在用的是bash还是zsh），写入一个程序

```shell
function jptt(){
    # Forwards port $1 into port $2 and listens to it
    ssh -N -f -L localhost:$2:localhost:$1 remoteuser@remotehost
    # ssh -N -f -L localhost:9906:localhost:9996 yanyuan@192.168.1.15
}
```

这样每次只需要在`PC`的terminal中输入如下命令，就可以打开Jupyter Lab的client端了

```shell
$ jptt 9996 9906
```

分别在`remote`和`PC`上source一下`.bashrc`(或`.zshrc`)让命令生效
```shell
$ source ~/.zshrc
```

然后打开`PC`上的浏览器，访问`http://localhost:9906/`
